## **1) Importing Python Packages for GAN**


In [ ]:
!nvidia-smi


Wed Jul 27 17:25:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam

import numpy as np
!mkdir generated_images

## **2) Variables for Neural Networks & Data**

In [ ]:
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(lr=0.0001)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## **3) Building Generator**





In [ ]:
# np.prod(img_shape)
28*28*1

784

In [ ]:
np.prod(img_shape)

784

In [ ]:
def build_generator():
  model = Sequential()

  model.add(Dense(256, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()
  return model

generator = build_generator()




Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 256)               25856     
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 256)               0         
                                                                 
 batch_normalization_15 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dense_20 (Dense)            (None, 256)               65792     
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 256)               0         
                                                                 
 batch_normalization_16 (Bat  (None, 256)              1024      
 chNormalization)                                     

In [ ]:
# def build_generator():
#   model = Sequential()

#   model.add(Dense(256, input_dim=latent_dim))
#   model.add(LeakyReLU(alpha=0.2))
#   model.add(BatchNormalization(momentum=0.8))

#   model.add(Dense(256))
#   model.add(LeakyReLU(alpha=0.2))
#   model.add(BatchNormalization(momentum=0.8))

#   model.add(Dense(256))
#   model.add(LeakyReLU(alpha=0.2))
#   model.add(BatchNormalization(momentum=0.8))

#   model.add(Dense(np.prod(img_shape), activation='tanh'))
#   model.add(Reshape(img_shape))
  
#   model.summary()
#   return model

# generator = build_generator()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 256)               25856     
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 256)               0         
                                                                 
 batch_normalization_11 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dense_17 (Dense)            (None, 256)               65792     
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 256)               0         
                                                                 
 batch_normalization_12 (Bat  (None, 256)              1024      
 chNormalization)                                     

## **4) Building Discriminator**

In [ ]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape))

  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  
  model.add(Dense(256))
  model.add(Dense(1, activation='sigmoid'))

  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_26 (Dense)            (None, 512)               401920    
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_27 (Dense)            (None, 256)               131328    
                                                                 
 dense_28 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# def build_discriminator():
#   model = Sequential()

#   model.add(Flatten(input_shape=img_shape))

#   model.add(Dense(512))
#   model.add(LeakyReLU(alpha=0.2))

#   model.add(Dense(256))
#   model.add(Dense(1, activation='sigmoid'))

#   model.summary()
#   return model

# discriminator = build_discriminator()
# discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

## **5) Connecting Neural Networks to build GAN**

In [ ]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)
GAN.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 28, 28, 1)         362000    
                                                                 
 sequential_8 (Sequential)   (None, 1)                 533505    
                                                                 
Total params: 895,505
Trainable params: 360,464
Non-trainable params: 535,041
_________________________________________________________________


## **6) Outputting Images**


In [ ]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    print("%.8f" % save_name)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            # axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    print('saved')
    plt.close()

## **7) Training GAN**

In [ ]:
def train(epochs, batch_size=64, save_interval=200):
  (X_train, _), (_, _) = mnist.load_data()
  print(X_train.shape)
  #Rescale data between -1 and 1
  X_train = X_train / 127.5 -1.
  print(X_train.shape)

  #Create our Y for our Neural Networks
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  #training loop
  for epoch in range(epochs):
    #Get Random Batch of data
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    #Generate Fake Images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    #Train discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    #take average 
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    
    #inverse y label
    g_loss = GAN.train_on_batch(noise, valid)

    print("******* %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100* d_loss[1], g_loss))

    if(epoch % save_interval) == 0:
      save_imgs(epoch)

  # print(valid)


train(30000, batch_size=64, save_interval=200)

Streaming output truncated to the last 5000 lines.
******* 669 [D loss: 0.043184, acc: 98.44%] [G loss: 4.784232]
******* 670 [D loss: 0.042196, acc: 99.22%] [G loss: 4.127063]
******* 671 [D loss: 0.102571, acc: 98.44%] [G loss: 4.381999]
******* 672 [D loss: 0.041783, acc: 99.22%] [G loss: 4.992873]
******* 673 [D loss: 0.033262, acc: 100.00%] [G loss: 4.737813]
******* 674 [D loss: 0.084325, acc: 97.66%] [G loss: 4.362000]
******* 675 [D loss: 0.150978, acc: 96.09%] [G loss: 3.183270]
******* 676 [D loss: 0.075901, acc: 99.22%] [G loss: 4.284227]
******* 677 [D loss: 0.038505, acc: 99.22%] [G loss: 4.855657]
******* 678 [D loss: 0.066302, acc: 97.66%] [G loss: 4.238800]
******* 679 [D loss: 0.115084, acc: 97.66%] [G loss: 3.336987]
******* 680 [D loss: 0.058550, acc: 100.00%] [G loss: 3.815915]
******* 681 [D loss: 0.077097, acc: 97.66%] [G loss: 4.369445]
******* 682 [D loss: 0.076135, acc: 97.66%] [G loss: 4.040741]
******* 683 [D loss: 0.107425, acc: 97.66%] [G loss: 3.642456]
**

### **8) Making GIF**

In [ ]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)